In [ ]:
import os
import sys
from pathlib import Path
from importlib import reload
import warnings
os.environ['USE_PYGEOS'] = '0'

import geopandas as gpd
from sqlalchemy import create_engine
import libpysal as lps
from esda import smoothing as sm
from libpysal.weights.distance import get_points_array
from scipy.spatial import cKDTree
from spreg import OLS
from esda import fdr
import spreg
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
from sklearn import metrics, preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler, minmax_scale
from sklearn.cluster import AgglomerativeClustering

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
from shapely.geometry import Point, Polygon
from matplotlib import patches, colors
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection
from matplotlib.colors import rgb2hex
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
from PIL import ImageColor
import matplotlib_inline.backend_inline
import pydotplus
import plotly.express as px
import pickle
from generativepy.color import Color
# from geoalchemy2 import Geometry, WKTElement
import contextily as ctx
from six import StringIO
from tableone import TableOne
# from statannotations.Annotator import Annotator

sys.path.append('/Users/david/Dropbox/PhD/Scripts/Spatial analyses')
import pyspace
from utils import *
engine = create_engine('postgresql://postgres@localhost:5432/david')
# Set pandas display options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Ignore this specific UserWarning
warnings.filterwarnings("ignore", category=UserWarning, message="The weights matrix is not fully connected")
warnings.filterwarnings("ignore", category=DeprecationWarning)
# Set matplotlib display format
# matplotlib_inline.backend_inline.set_matplotlib_formats('retina')
%matplotlib inline

In [ ]:
import esda

In [ ]:
#Set working directory
mydir = Path(os.getcwd())
data_folder = mydir / '../Data' # Set data folder
result_folder = mydir / '../Results' # Set results folder
figures_folder = Path('../Manuscript/Figures - EN/') # Set figures folder

## Import data
### BCS participation

In [ ]:
# Participation data
df_participation = pd.read_feather('../Data/Processed data/20230627_GIRACS_all.feather')
gdf_participation = gpd.read_feather('../Data/Processed data/20230627_GIRACS_all.feather')

print('Number of invitations :', gdf_participation.shape[0])
print('Number of women :', gdf_participation.numerodossier.nunique())

In [ ]:
gdf_participation[(gdf_participation['year_invit']>2002)].numerodossier.nunique()

### Administrative units

In [ ]:
# Administrative boundaries
lake = gpd.read_file('/Users/david/Dropbox/PhD/GitHub/COVID19/Data/Mapping/lake.geojson')
lake.NOM = ['Lake Geneva', '', '']
cantons = gpd.read_file(
    '/Users/david/Dropbox/PhD/Data/Databases/SITG/SHAPEFILE_LV95_LN02/swissBOUNDARIES3D_1_3_TLM_KANTONSGEBIET.shp')
communes_ch = gpd.read_file(
    '/Users/david/Dropbox/PhD/Data/Databases/SITG/SHAPEFILE_LV95_LN02/swissBOUNDARIES3D_1_3_TLM_HOHEITSGEBIET.shp')
# Only retain communes that are in the canton of Geneva
communes = communes_ch[communes_ch.KANTONSNUM == 25]

cantons = cantons.to_crs(2056)
communes_ch = communes_ch.to_crs(2056)
communes = communes.to_crs(2056)

## Study setting

In [ ]:
import matplotlib.patches as patches
from matplotlib.patches import ConnectionPatch
import numpy as np
from shapely.geometry import box
import contextily as ctx
from matplotlib_scalebar.scalebar import ScaleBar

In [ ]:
communes[communes.NAME=='Genève'].geometry.centroid

In [ ]:
def create_geneva_context_map(cantons, communes):
    """
    Create a context map showing Geneva canton within Switzerland,
    with an inset map showing Geneva's municipalities.
    
    Parameters:
    -----------
    cantons : geopandas.GeoDataFrame
        GeoDataFrame containing all Swiss cantons with 'NAME' column
    communes : geopandas.GeoDataFrame  
        GeoDataFrame containing Geneva municipalities with 'NAME' column
    """
    
    # === DATA PREPARATION ===
    # Ensure both GeoDataFrames are in WGS84 (EPSG:4326) for plotting
    # if cantons.crs != 'EPSG:4326':
    #     cantons = cantons.to_crs('EPSG:4326')
    # if communes.crs != 'EPSG:4326':
    #     communes = communes.to_crs('EPSG:4326')
    fig = plt.figure(figsize=(12, 8))
    # Extract Geneva canton
    geneva_canton = cantons[cantons['NAME'] == 'Genève'].copy()
    if geneva_canton.empty:
        raise ValueError("Geneva canton not found. Check that 'NAME' column contains 'Genève'")
    
    # Extract Geneva city (most populated municipality)
    geneva_city = communes[communes['NAME'] == 'Genève'].copy()
    if geneva_city.empty:
        print("Warning: Geneva city not found in communes dataset")
        geneva_city = None
    
    # Main Switzerland map (left side, larger)
    ax_main = plt.subplot2grid((2, 3), (0, 0), colspan=2, rowspan=2)
    
    # Geneva detail map (right side, smaller)
    ax_detail = plt.subplot2grid((2, 4), (0, 3), rowspan=1)
    
    # === MAIN SWITZERLAND MAP ===
    # ax_main.set_title('Switzerland - Canton of Geneva Location', 
    #                  fontsize=14, fontweight='bold', pad=20)
    
    # Plot all Swiss cantons
    cantons.plot(ax=ax_main, color='lightgray', edgecolor='white', linewidth=0.5)
    
    # Highlight Geneva canton
    geneva_canton.plot(ax=ax_main, color='red', alpha=0.7, edgecolor='darkred', linewidth=2)
    
    # Get Geneva bounds for boundary rectangle and connection
    geneva_bounds = geneva_canton.total_bounds  # [minx, miny, maxx, maxy]
    
    # Add boundary rectangle around Geneva for connection
    boundary_rect = patches.Rectangle((geneva_bounds[0], geneva_bounds[1]), 
                                     geneva_bounds[2] - geneva_bounds[0],
                                     geneva_bounds[3] - geneva_bounds[1],
                                     linewidth=2, edgecolor='black', 
                                     facecolor='none', linestyle='-')
    ax_main.add_patch(boundary_rect)
    
    # Set appropriate limits for Switzerland
    swiss_bounds = cantons.total_bounds
    ax_main.set_xlim(swiss_bounds[0] - 0.1, swiss_bounds[2] + 0.1)
    ax_main.set_ylim(swiss_bounds[1] - 0.1, swiss_bounds[3] + 0.1)
    ax_main.set_xlabel('Easting', fontsize=12)
    ax_main.set_ylabel('Northing', fontsize=12)
    ax_main.grid(True, alpha=0.3)
    
    # Add major Swiss cities for reference (approximate coordinates)
    cities = {
        'Bern': [2597542.079, 1199612.021],
        'Zürich': [2682517.95, 1248415.856],
        'Basel': [2611664.667, 1267460.265],
        'Geneva': [2500062.992, 1118117.691]
    }
    
    for city, coords in cities.items():
        ax_main.plot(coords[0], coords[1], 'ko', markersize=6)
        ax_main.annotate(city, coords, xytext=(5, -15), 
                        textcoords='offset points', fontsize=10,
                        bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
    
    # === GENEVA DETAIL MAP ===
    # ax_detail.set_title('Canton of Geneva\nMunicipalities', 
    #                    fontsize=12, fontweight='bold', pad=15)
    
    # Plot all Geneva municipalities
    communes.plot(ax=ax_detail, color='lightblue', 
                 edgecolor='navy', linewidth=1, alpha=0.7)
    
    # Highlight Geneva city (most populated municipality) if found
    if geneva_city is not None:
        geneva_city.plot(ax=ax_detail, color='orange', 
                        edgecolor='darkorange', linewidth=2, alpha=0.8)
    
    # Add municipality labels for the largest/most important ones
    # Calculate centroids for labeling
    communes_with_centroids = communes.copy()
    communes_with_centroids['centroid'] = communes_with_centroids.geometry.centroid
    communes_with_centroids['x'] = communes_with_centroids.centroid.x
    communes_with_centroids['y'] = communes_with_centroids.centroid.y
    
    # Label key municipalities (you can customize this list)
    key_municipalities = ['Genève']
    
    for idx, row in communes_with_centroids.iterrows():
        if row['NAME'] in key_municipalities:
            ax_detail.annotate('Geneva', (row['x'], row['y']), 
                             fontsize=8, ha='center', va='center',
                             bbox=dict(boxstyle='round,pad=0.2', 
                                     facecolor='white', alpha=0.8))
    
    # Set limits to Geneva bounds with small buffer
    buffer = 0.01
    ax_detail.set_xlim(geneva_bounds[0] - buffer, geneva_bounds[2] + buffer)
    ax_detail.set_ylim(geneva_bounds[1] - buffer, geneva_bounds[3] + buffer)
    ax_detail.set_xlabel('Easting', fontsize=10)
    ax_detail.set_ylabel('Northing', fontsize=10)
    ax_detail.grid(True, alpha=0.3)
                
    # Add basemap
    # ctx.add_basemap(ax_detail, crs=communes.crs, source=ctx.providers.OpenStreetMap.Mapnik, alpha=0.5)
    # add_north_arrow(ax_main)
    # add_north_arrow(ax_detail)
    add_scale_bar(ax_main)
    add_scale_bar(ax_detail)

    # === CONNECTION LINE ===
    # Create connection line from main map boundary to detail map
    # Get the connection points
    geneva_center = [geneva_bounds[0] + (geneva_bounds[2] - geneva_bounds[0])/2, 
                    geneva_bounds[1] + (geneva_bounds[3] - geneva_bounds[1])/2]
    
    main_connection_point = (geneva_bounds[2], geneva_center[1])  # Right side of Geneva rectangle
    detail_connection_point = (geneva_bounds[0], geneva_center[1])  # Left side of detail map
    
    # Create connection patch
    con = ConnectionPatch(main_connection_point, detail_connection_point,
                         "data", "data", axesA=ax_main, axesB=ax_detail,
                         arrowstyle="->", shrinkB=5, shrinkA=5,
                         color="black", linewidth=1.5)
    ax_detail.add_artist(con)
    
    # === STYLING AND LAYOUT ===
    plt.tight_layout()
    
    # Add overall title
    # fig.suptitle('Canton of Geneva in Switzerland - Study Area Context', 
    #             fontsize=16, fontweight='bold', y=0.95)
    
    # Add scale bars and north arrows if needed
    # (You can add these using matplotlib-scalebar or custom functions)
    
    # Add legend
    legend_elements = [
        patches.Patch(color='red', alpha=0.7, label='Canton of Geneva'),
        patches.Patch(color='lightblue', alpha=0.7, label='Geneva municipalities'),
        patches.Patch(color='orange', alpha=0.8, label='City of Geneva'),
        plt.Line2D([0], [0], color='black', linestyle='-', label='Study area boundary')
    ]
    ax_main.legend(handles=legend_elements, loc='upper right', 
                  bbox_to_anchor=(1.56, 0.5), fontsize=12)
    
    return fig, (ax_main, ax_detail)
def add_north_arrow(ax, location=(0.9, 0.9)):
    """
    Add a north arrow to the map (optional enhancement).
    
    Parameters:
    -----------
    ax : matplotlib.axes.Axes  
        The axes to add the north arrow to
    location : tuple
        Location of north arrow as (x, y) in axes coordinates
    """
    # Add a simple north arrow
    ax.annotate('N', xy=location, xycoords='axes fraction',
                fontsize=16, fontweight='bold', ha='center', va='center',
                bbox=dict(boxstyle='circle', facecolor='white', edgecolor='black'))
    ax.annotate('↑', xy=(location[0], location[1]-0.03), xycoords='axes fraction',
                fontsize=14, ha='center', va='center')

def add_scale_bar(ax):
    scalebar = ScaleBar(1, units="m", location="lower right")
    ax.add_artist(scalebar)

In [ ]:
fig, (ax_main, ax_detail) = create_geneva_context_map(cantons, communes)
plt.savefig(result_folder/'geneva_context_map.png', dpi=300, bbox_inches='tight')

### BCS screening centers

In [ ]:
gdf_centre = gpd.read_file('../Data/Raw data/BC_ScreeningCenters.geojson',driver = 'GeoJSON')

gdf_centre.crs = 4326

gdf_centre['geometry'] = gpd.points_from_xy(gdf_centre.lon, gdf_centre.lat)

gdf_centre = gdf_centre.to_crs(2056)

gdf_centre['year_start'] = gdf_centre['year_start'].astype(int)

## Evolution of BCS participation

In [ ]:
gdf_participation.loc[gdf_participation['year_invit'].isin([1999,2000]), 'year_invit_2y'] = '2000'
gdf_participation.loc[gdf_participation['year_invit'].isin([2001,2002]), 'year_invit_2y'] = '2002'
gdf_participation.loc[gdf_participation['year_invit'].isin([2003,2004]), 'year_invit_2y'] = '2004'
gdf_participation.loc[gdf_participation['year_invit'].isin([2005,2006]), 'year_invit_2y'] = '2006'
gdf_participation.loc[gdf_participation['year_invit'].isin([2007,2008]), 'year_invit_2y'] = '2008'
gdf_participation.loc[gdf_participation['year_invit'].isin([2009,2010]), 'year_invit_2y'] = '2010'
gdf_participation.loc[gdf_participation['year_invit'].isin([2011,2012]), 'year_invit_2y'] = '2012'
gdf_participation.loc[gdf_participation['year_invit'].isin([2013,2014]), 'year_invit_2y'] = '2014'
gdf_participation.loc[gdf_participation['year_invit'].isin([2015,2016]), 'year_invit_2y'] = '2016'
gdf_participation.loc[gdf_participation['year_invit'].isin([2017,2018]), 'year_invit_2y'] = '2018'
gdf_participation.loc[gdf_participation['year_invit'].isin([2019,2020]), 'year_invit_2y'] = '2020'

In [ ]:
gdf_participation.groupby('year_invit_2y').size().to_clipboard()

In [ ]:
gdf_participation[gdf_participation.mammo==1].groupby('year_invit_2y').size().to_clipboard()

In [ ]:
n_invit_by_year = pd.DataFrame(gdf_participation.groupby('year_invit_2y').size()).reset_index()
pal = sns.color_palette("Blues", len(n_invit_by_year))
rank = n_invit_by_year[0].argsort().argsort()   # http://stackoverflow.com/a/6266510/1628638

fig, ax = plt.subplots(1, 1, figsize=(8, 8))
chart = sns.barplot(x="year_invit_2y", y=0, data=n_invit_by_year, hue='year_invit_2y', palette = np.array(pal)[rank], legend=False, ax = ax)
chart.set_xticklabels(chart.get_xticklabels(),size = 8, rotation=45, horizontalalignment='right')
chart.set_xlabel("Biennal period of invitation",size = 12)
chart.set_ylabel("Invitation count",size = 12)

In [ ]:
n_mammo_by_year = pd.DataFrame(gdf_participation.groupby('year_invit_2y').mammo.sum()).reset_index()
pal = sns.color_palette("Blues", len(n_mammo_by_year))
rank = n_mammo_by_year['mammo'].argsort().argsort()   # http://stackoverflow.com/a/6266510/1628638

fig, ax = plt.subplots(1, 1, figsize=(8, 8))
chart = sns.barplot(x="year_invit_2y", y="mammo", data=n_mammo_by_year, hue='year_invit_2y', palette = np.array(pal)[rank], legend=False, ax = ax)
chart.set_xticklabels(chart.get_xticklabels(),size = 10, rotation=45, horizontalalignment='right')
chart.set_xlabel("Biennal period of invitation",size = 12)
chart.set_ylabel('Mammography count',size = 12)
show_values(chart, digits = 0, fontsize = 7)

In [ ]:
raw_rate_by_year = pd.DataFrame(n_mammo_by_year.set_index('year_invit_2y')['mammo'].div(n_invit_by_year.set_index('year_invit_2y')[0]).mul(100)).reset_index()

pal = sns.color_palette("Blues", len(n_mammo_by_year))
rank = raw_rate_by_year[0].argsort().argsort()   # http://stackoverflow.com/a/6266510/1628638

fig, ax = plt.subplots(1, 1, figsize=(8, 8))
chart = sns.barplot(x="year_invit_2y", y=0, data=raw_rate_by_year, hue='year_invit_2y', palette = np.array(pal)[rank], legend=False, ax = ax)
chart.set_xticklabels(chart.get_xticklabels(),size = 10, rotation=45, horizontalalignment='right')
chart.set_xlabel("Biennal period of invitation",size = 12)
chart.set_ylabel('Participation rate (%)',size = 12)
show_values(chart)

In [ ]:
n_mammo_by_year[n_mammo_by_year.year_invit_2y.isin(['2000','2002'])==False].mammo.sum()

In [ ]:
gdf_participation[gdf_participation.year_invit_2y.isin(['2000','2002'])==False].groupby('year_invit_2y').mammo.sum().sum()

In [ ]:
gdf_participation[gdf_participation.year_invit_2y.isin(['2000','2002'])==False].numerodossier.nunique()

In [ ]:
gdf_participation[(gdf_participation.year_invit_2y.isin(['2000','2002'])==False)&(gdf_participation.mammo==1)&(gdf_participation.mammoanterieure=='non')].groupby('year_invit_2y')['numerodossier'].nunique()

## Final data processing

In [ ]:
gdf_participation = pd.merge(gdf_participation,pd.get_dummies(gdf_participation['etatcivil']), left_index = True, right_index = True)

In [ ]:
gdf_participation['deprivation_pca_q5_txt'] = gdf_participation['deprivation_pca_q5'].map({0:"0 - Least deprived", 1:'1',2:'2',3:'3', 4: '4 - Most deprived'})
gdf_participation['deprivation_pca_q5_txt'] = pd.Categorical(gdf_participation['deprivation_pca_q5_txt'], ordered = True, categories = ['0 - Least deprived','1','2','3','4 - Most deprived'])

In [ ]:
# Drop duplicates caused by spatial join (some individuals intersects with multiple ha)
gdf_participation = gdf_participation[~gdf_participation.uuid.duplicated()]

In [ ]:
n_mammo_by_2year = pd.DataFrame(gdf_participation.groupby('year_invit_2y').mammo.sum()).reset_index()
n_invit_by_2year = pd.DataFrame(gdf_participation.groupby('year_invit_2y').size()).reset_index()
raw_rate_by_2year = pd.DataFrame(n_mammo_by_2year.set_index('year_invit_2y')['mammo'].div(n_invit_by_2year.set_index('year_invit_2y')[0]), columns = ['raw rate']).reset_index()
# Calculate standard errors
raw_rate_by_2year['se'] = np.sqrt(raw_rate_by_2year['raw rate'] * (1 - raw_rate_by_2year['raw rate']) / n_invit_by_2year[0])

In [ ]:
n_mammo_by_2year_age = pd.DataFrame(gdf_participation.groupby(['year_invit_2y','deprivation_pca_q5']).mammo.sum()).reset_index()
n_invit_by_2year_age = pd.DataFrame(gdf_participation.groupby(['year_invit_2y','deprivation_pca_q5']).size()).reset_index()
raw_rate_by_2year_age = pd.DataFrame(n_mammo_by_2year_age.set_index(['year_invit_2y','deprivation_pca_q5'])['mammo'].div(n_invit_by_2year_age.set_index(['year_invit_2y','deprivation_pca_q5'])[0]), columns = ['raw rate']).reset_index()
# Calculate standard errors
raw_rate_by_2year_age['se'] = np.sqrt(raw_rate_by_2year_age['raw rate'] * (1 - raw_rate_by_2year_age['raw rate']) / n_invit_by_2year_age[0])

In [ ]:
raw_rate_by_2year_age[['raw rate','se']] = raw_rate_by_2year_age[['raw rate','se']].mul(100)

In [ ]:
gdf_participation['year_invit_2y_int'] = gdf_participation['year_invit_2y'].astype(int)
gdf_participation['year_invit_2y_int'] = pd.to_datetime(gdf_participation['year_invit_2y_int'], format = '%Y')
gdf_participation['mammo_100'] = gdf_participation['mammo'] *100

In [ ]:
# _ha = pd.DataFrame(gdf_participation_ha.groupby(['RELI','year_invit']).mammo.sum()).reset_index()
n_mammo_by_neighborhood = pd.DataFrame(gdf_participation.groupby(['nbid','year_invit_2y']).mammo.sum()).reset_index()
n_invit_by_neighborhood = pd.DataFrame(gdf_participation.groupby(['nbid','year_invit_2y']).size()).reset_index()

n_mammo_by_neighborhood.columns = ['nbid','year_invit','n_mammo']
n_invit_by_neighborhood.columns = ['nbid','year_invit','n_invit']

#f.pivot(index='foo', columns='bar', values='baz')
n_mammo_by_neighborhood = n_mammo_by_neighborhood.pivot(index ='nbid',columns = 'year_invit',values = 'n_mammo').fillna(0).reset_index()
n_invit_by_neighborhood = n_invit_by_neighborhood.pivot(index ='nbid',columns = 'year_invit',values = 'n_invit').fillna(0).reset_index()

## Final number checks

In [ ]:
gdf_participation[gdf_participation.mammo > 0].numerodossier.nunique()

In [ ]:
print("Overall mammography (> 0) rate is : ",round(gdf_participation[gdf_participation.numerodepistage > 0].numerodossier.nunique()/gdf_participation.numerodossier.nunique(), 3))

In [ ]:
print("Overall participation rate is : ",round(gdf_participation[gdf_participation.mammo==1].shape[0]/gdf_participation.shape[0], 3))

## Temporal trends

In [ ]:
# Increase the figure size
plt.figure(figsize=(6, 6))

# Use a custom color palette

sns.lineplot(x='year_invit_2y_int', y='mammo_100', data=gdf_participation[gdf_participation.year_invit_2y.isin(['2000','2002'])==False])
markers = pd.DataFrame(gdf_participation[gdf_participation.year_invit_2y.isin(['2000','2002'])==False].groupby('year_invit_2y').mammo_100.mean()).reset_index()
markers['year_invit_2y'] = markers['year_invit_2y'].astype('int32')
# Add a title
# plt.title('Participation Rate Evolution Over Time Stratified By SES Deprivation Index', fontsize=16)


# Increase the font size of the labels
plt.xlabel('Biennial interval', fontsize=12)
plt.ylabel('Participation rate (%)', fontsize=12)


# Setting x-ticks every 2 years
years = mdates.YearLocator(2) 
ax = plt.gca()
ax.xaxis.set_major_locator(years)

# Add a grid to both axes
ax.grid(True, color='grey', linestyle='--', linewidth=0.5, zorder=0)

# Format x-tick labels as YYYY
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

ax.axvline(pd.to_datetime('2014-01-01'), color='darkgrey', linestyle='--', lw=1.4, zorder=0)  # Change to the exact date if needed
ax.text(pd.to_datetime('2014-01-15'), ax.get_ylim()[1]*0.7, ' Program expanded to\n women aged 70-74', color='grey')

# Save the figure with a high resolution
# plt.savefig('../Manuscript/Figures - EN/Bar plot - Participation rate by 2y - Neighborhood.png', dpi = 300, bbox_inches='tight')

# Show the plot
plt.show()

In [ ]:
ax.get_ylim()[1]

In [ ]:
# Increase the figure size
plt.figure(figsize=(6, 6))

# Use a custom color palette
palette = sns.color_palette("RdYlGn_r", 5)

sns.lineplot(x='year_invit_2y_int', y='mammo_100', hue='deprivation_pca_q5_txt', palette=palette, data=gdf_participation[gdf_participation.year_invit_2y.isin(['2000','2002'])==False], alpha=1, zorder=4)

# Add a title
# plt.title('Participation Rate Evolution Over Time Stratified By SES Deprivation Index', fontsize=16)

# Increase the font size of the labels
plt.xlabel('Biennial interval', fontsize=12)
plt.ylabel('Participation rate (%)', fontsize=12)

# Increase the font size of the legend and its title
plt.legend(title='SES deprivation index (quintiles)', title_fontsize='13', fontsize='12')

# Setting x-ticks every 2 years
years = mdates.YearLocator(2) 
ax = plt.gca()
ax.xaxis.set_major_locator(years)

# Add a grid to both axes
ax.grid(True, color='grey', linestyle='--', linewidth=0.5, zorder=0)

# Format x-tick labels as YYYY
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Save the figure with a high resolution
# plt.savefig('../Manuscript/Figures - EN/Bar plot - Participation rate by 2y and SES - Neighborhood.png', dpi = 300, bbox_inches='tight')

# Show the plot
plt.show()

In [ ]:
gdf_participation['groupeage'] = gdf_participation['groupeage'].astype('str')

In [ ]:
plt.figure(figsize=(6, 6))
palette = sns.color_palette("Greens", 5)

chart = sns.lineplot(x='year_invit_2y_int', y='mammo_100', hue='groupeage', data=gdf_participation[gdf_participation.year_invit_2y.isin(['2000','2002'])==False])
show_values(chart)
# plt.title('Participation Rate Evolution Over Time Stratified By Age Group')
plt.xlabel('Biennial interval', fontsize=12)
plt.ylabel('Participation rate (%)', fontsize=12)
plt.legend(title='Age group (years)', loc = 'lower right')
# setting x-ticks every 2 years
years = mdates.YearLocator(2) 
ax = plt.gca()
ax.xaxis.set_major_locator(years)
ax.grid(True, color='grey', linestyle='--', linewidth=0.5, zorder=0)
# format x-tick labels as YYYY
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
# plt.savefig('../Manuscript/Figures - EN/Bar plot - Participation rate by 2y and age group - Neighborhood.png', dpi = 300, bbox_inches='tight')
plt.show()

## Data Aggregation at the Swiss Neighborhood scale

In [ ]:
microgis_data = gpd.read_parquet(data_folder/'Processed data'/'microgis_data_depriv.parquet')

In [ ]:
microgis_data = microgis_data[['nbid', 'ptot', 'ciqmd','deprivation_pca', 'deprivation_pca_q5','dmdrent','wo_tertiary_education','rpnch','radune','rad3tert', 'geometry']]

In [ ]:
nbid_deprivation = microgis_data.set_index('nbid')['deprivation_pca'].to_dict()

In [ ]:
# Considering all invitations
gdf_neighborhood = pd.DataFrame(gdf_participation.nbid.unique(), columns = ['nbid'])
gdf_neighborhood = pd.merge(microgis_data, n_invit_by_neighborhood, on = 'nbid').merge(n_mammo_by_neighborhood, on = 'nbid', suffixes = ('_invit','_mammo'))

In [ ]:
gdf_neighborhood['index_socio_q3'] = pd.qcut(gdf_neighborhood['deprivation_pca'], 3, labels = False)
gdf_neighborhood.loc[gdf_neighborhood['index_socio_q3'] == 0,'index_socio_q3'] = 'Low'
gdf_neighborhood.loc[gdf_neighborhood['index_socio_q3'] == 1,'index_socio_q3'] = 'Medium'
gdf_neighborhood.loc[gdf_neighborhood['index_socio_q3'] == 2,'index_socio_q3'] = 'High'

In [ ]:
gdf_neighborhood['index_socio_q3'] = pd.Categorical(gdf_neighborhood['index_socio_q3'],categories = ['Low','Medium','High'], ordered = True)

In [ ]:
dict_cl = {np.nan:'#bababa',"High":'#d7191c',
3:'#fdae61',
'Medium':'#ffffbf',
1:'#abd9e9',
"Low":'#2c7bb6'}
hmap = colors.ListedColormap([dict_cl[i] for i in gdf_neighborhood['index_socio_q3'].sort_values().unique()])

In [ ]:
# DROP 2000 and 2002 DATA TO BE SURE THEY AREN'T USED
gdf_neighborhood = gdf_neighborhood.drop(['2000_invit','2002_invit','2000_mammo','2002_mammo'],axis=1)

In [ ]:
gdf_neighborhood['2006_invit'].plot.hist(bins=50)
plt.show()

## Mapping raw and adjusted rates

In [ ]:
providers = ctx.providers.flatten()

In [ ]:
dfs_year = []
for year in range(2004,2022,2):
    invit_year = str(year)+'_invit'
    mammo_year = str(year)+'_mammo'

    df_neighborhood_year = gdf_neighborhood[gdf_neighborhood[invit_year] > 0][['nbid',invit_year,mammo_year,'geometry']]
    wnn16 = lps.weights.KNN(cKDTree(get_points_array(df_neighborhood_year.geometry.centroid)), 16)
    wnn8 = lps.weights.KNN(cKDTree(get_points_array(df_neighborhood_year.geometry.centroid)), 8)
    # wnn4 = lps.weights.KNN(cKDTree(get_points_array(df_neighborhood_year.geometry.centroid)), 4)

    e = np.array(df_neighborhood_year[mammo_year])
    b = np.array(df_neighborhood_year[invit_year])
    ebs_rate = sm.Spatial_Empirical_Bayes(e, b, wnn8)
    risk = sm.Excess_Risk(e, b)
    df_neighborhood_year['tx_screening'] = df_neighborhood_year[mammo_year].div(df_neighborhood_year[invit_year]).mul(100)
    df_neighborhood_year['ebs_rate'] = ebs_rate.r
    df_neighborhood_year['ebs_rate'] = df_neighborhood_year['ebs_rate'].fillna(0)
    df_neighborhood_year['ebs_rate'] = df_neighborhood_year['ebs_rate'].mul(100)
    df_neighborhood_year['excess_risk'] = risk.r
    df_neighborhood_year['excess_risk'] = df_neighborhood_year['excess_risk'].fillna(0)
    df_neighborhood_year['er_bins'] = pd.cut(df_neighborhood_year['excess_risk'], bins = [0,0.25,0.5,0.75,1,1.25,2,4,10])
    df_neighborhood_year['ebs_bins'] = pd.cut(df_neighborhood_year['ebs_rate'], bins = [0,10,20,30,40,50,60,80,100])
    df_neighborhood_year['diff_rawrate_vs_ebsrate'] = df_neighborhood_year['tx_screening']-(df_neighborhood_year['ebs_rate'])
    dfs_year.append(df_neighborhood_year)

In [ ]:
year = 2004
for df_year in dfs_year:
    fig, ax = plt.subplots(figsize = (15,15))

    df_year.plot('tx_screening',cmap = 'Blues', linewidth = 0.01, scheme = 'quantiles', legend = True,legend_kwds = {'title': "Taux de participation (brut) (%)",'fmt':"{:.1f}",'loc': 'upper left'},ax = ax)
    communes.geometry.boundary.plot(linewidth = 0.3, color = 'grey', facecolor = None,ax = ax, zorder =0)
    lake.plot(color = 'lightblue',ax = ax)
    # ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.HOT,crs = 'EPSG:2056')
    lake.apply(lambda x: ax.annotate(text=x.NOM, xy=x.geometry.centroid.coords[0], ha='center',size = 8),axis=1);
    # communes.apply(lambda x: ax.annotate(text=x.NAME, xy=x.geometry.centroid.coords[0], ha='center',size = 6),axis=1);

    # add scale bar
    scalebar = ScaleBar(1, units="m", location="lower right")
    ax.add_artist(scalebar)
    ax.set_title('Raw participation rates to the breast cancer screening program %s'% str(year))
    # ax.set_facecolor('black')
    ax.set_axis_off()
    filename = '%s.png' % str(year)
    filepath = result_folder/'Rate maps'/'Raw'/filename
    # plt.savefig(filepath,dpi = 300, bbox_inches='tight')
    plt.clf()
    plt.close(fig)
    
    year += 2

In [ ]:
year = 2004
for df_year in dfs_year:
    fig, ax = plt.subplots(figsize = (15,15))
    # ax = communes.plot(color = 'darkgrey',figsize = (15,15))

    df_year.plot('ebs_rate',cmap = 'Blues', linewidth = 0.01, scheme = 'quantiles', legend = True,legend_kwds = {'title': "Taux de participation (EBS) (%)",'fmt':"{:.1f}",'loc': 'upper left'},ax = ax)
    communes.geometry.boundary.plot(linewidth = 0.3, color = 'grey', facecolor = None,ax = ax, zorder =0)

    lake.plot(color = 'lightblue',ax = ax)
    # ctx.add_basemap(ax, source=ctx.providers.Stadia.StamenTonerLite,crs = 2056)
    lake.apply(lambda x: ax.annotate(text=x.NOM, xy=x.geometry.centroid.coords[0], ha='center',size = 8),axis=1);
    communes.apply(lambda x: ax.annotate(text=x.NAME, xy=x.geometry.centroid.coords[0], ha='center',size = 6),axis=1);

    # add scale bar
    scalebar = ScaleBar(1, units="m", location="lower right")
    ax.add_artist(scalebar)
    ax.set_title('SEBS participation rates to the breast cancer screening program in %s'% str(year))
    # ax.set_facecolor('black')
    ax.set_axis_off()
    filename = '%s.png' % str(year)
    filepath = result_folder/'Rate maps'/'Empirical Bayes Smoothing'/filename
    # plt.savefig(filepath,dpi = 300, bbox_inches='tight')
    plt.clf()
    plt.close(fig)
    
    year += 2

## Getis analyses - SEBS rates

In [ ]:
start_year = 2004
for df in dfs_year:
    # Create K-nearest neighbors weights object using the centroids of geometries
    weights_knn = lps.weights.KNN(cKDTree(get_points_array(df.geometry.centroid)), 16)

    # Compute Getis Ord's statistics
    getis_ord_stats = pyspace.compute_getis(df, 'ebs_rate', weights_knn, n_permut=9999, transform_type='B', p_001=True)

    # Adjust p-values using the False Discovery Rate method
    fdr_adjusted_p_value = fdr(getis_ord_stats.p_sim, alpha=0.05)
    
    # Create a new column for Getis Ord's classification with FDR adjustment
    df['ebs_rate_G_cl_fdr'] = df['ebs_rate_G_cl'] 
    df.loc[df['ebs_rate_G_psim'] >= fdr_adjusted_p_value, 'ebs_rate_G_cl_fdr'] = 'Not significant'

    # Plot Getis Ord's map
    fig, ax = pyspace.plotGetisMap_ge(df, 'ebs_rate_G_cl', p_001=True, commune_name=False)
    ax.set_title(start_year)

    # Save the plot
    filename = f'getis_giracs_ha_wnn16_005_{start_year}.png'
    # plt.savefig(result_folder / 'Getis' / filename, dpi=300, bbox_inches='tight')

    # Increase the start year by 2 for the next iteration
    start_year += 2

In [ ]:
colors_getis = ['#1c4978', '#2166ac', '#67a9cf', '#d1e5f0', '#991d2c', '#b2182b', '#ef8a62', '#fddbc7', '#c4bebe']
colors_cl_getis = {'Cold Spot - p < 0.001':'#1c4978',
                    'Cold Spot - p < 0.01':'#2166ac',
                    'Cold Spot - p < 0.05':'#67a9cf',
                    'Cold Spot - p < 0.1':'#d1e5f0',
                    'Hot Spot - p < 0.001':'#991d2c',
                    'Hot Spot - p < 0.01':'#b2182b',
                    'Hot Spot - p < 0.05':'#ef8a62',
                    'Hot Spot - p < 0.1':'#fddbc7',
                    'Not significant':'#f7f7f7'}
colors_cl = {'0 Non-Significant': 'lightgrey',
             '1 High-High': '#d7191c',
             '2 Low-High': '#abd9e9',
             '3 Low-Low': '#2c7bb6',
             '4 High-Low': '#fdae61'}

In [ ]:
fig, ax = plt.subplots(figsize = (6, 6))
chart = sns.barplot(x = 'ebs_rate_G_cl', y = 'ebs_rate', data = df_year, order=
    ['Cold Spot - p < 0.001','Cold Spot - p < 0.01', 'Cold Spot - p < 0.05', 'Cold Spot - p < 0.1','Hot Spot - p < 0.001', 'Hot Spot - p < 0.01',
     'Hot Spot - p < 0.05', 'Hot Spot - p < 0.1', 'Not significant'],
                        palette=colors_getis, errorbar=('ci', 95),  ax = ax)
chart.set_xlabel('Getis Classes', size=12)
chart.set_ylabel('SEBS participation rates (%)', size=12)
chart.set_xticklabels(chart.get_xticklabels(), size=8, rotation=45, horizontalalignment='right')

plt.title('Average SEBS Participation Rates (%) by Getis Class', size = 12)
# plt.savefig(figures_folder/'Bar plot - GIRACS -  Getis 2020.png', dpi = 300, bbox_inches = 'tight')
plt.show()

## Raw rates

In [ ]:
start_year = 2004
for df in dfs_year:
    # Create K-nearest neighbors weights object using the centroids of geometries
    weights_knn = lps.weights.KNN(cKDTree(get_points_array(df.geometry.centroid)), 16)

    # Compute Getis Ord's statistics
    getis_ord_stats = pyspace.compute_getis(df, 'tx_screening', weights_knn, n_permut=9999, transform_type='B', p_001=True)

    # Adjust p-values using the False Discovery Rate method
    fdr_adjusted_p_value = fdr(getis_ord_stats.p_sim, alpha=0.05)
    
    # Create a new column for Getis Ord's classification with FDR adjustment
    df['tx_screening_G_cl_fdr'] = df['tx_screening_G_cl'] 
    df.loc[df['tx_screening_G_psim'] >= fdr_adjusted_p_value, 'tx_screening_G_cl_fdr'] = 'Not significant'

    # Plot Getis Ord's map
    fig, ax = pyspace.plotGetisMap_ge(df, 'tx_screening_G_cl', p_001=True, commune_name=False)
    ax.set_title(start_year)

    # Save the plot
    filename = f'getis_giracs_ha_wnn16_005_raw_{start_year}.png'
    # plt.savefig(result_folder / 'Getis' / filename, dpi=300, bbox_inches='tight')

    # Increase the start year by 2 for the next iteration
    start_year += 2

## Temporal evolution of participation rate

In [ ]:
df_all_years = pd.DataFrame()
year = 2004
for df_year in dfs_year:
    df_year['year_invit'] = str(year)
    df_all_years = pd.concat([df_all_years, df_year])
    year +=2

In [ ]:
nbid_2020 = df_all_years[df_all_years['2020_mammo'].isnull()==False].nbid.tolist()

df_all_years_2020_neighborhood = df_all_years[(df_all_years.nbid.isin(nbid_2020))]

In [ ]:
df_all_years_2020_neighborhood  = df_all_years_2020_neighborhood.set_index('nbid')
df_all_years_2020_neighborhood = df_all_years_2020_neighborhood.reset_index()

In [ ]:
fig, ax = plt.subplots(figsize = (15,8))
# the size of A4 paper
g = sns.lineplot(data=df_all_years_2020_neighborhood, x="year_invit", y="ebs_rate", hue = 'ebs_rate_G_cl',hue_order=
    ['Cold Spot - p < 0.001','Cold Spot - p < 0.01', 'Cold Spot - p < 0.05', 'Cold Spot - p < 0.1','Hot Spot - p < 0.001', 'Hot Spot - p < 0.01',
     'Hot Spot - p < 0.05', 'Hot Spot - p < 0.1', 'Not significant'], palette = colors_cl_getis,ax = ax)
for l in g.lines:
    y = l.get_ydata()
    if len(y)>0:
        g.annotate(f'{y[-1]:.1f}', xy=(0.97,y[-1]), xycoords=('axes fraction', 'data'), 
                     ha='left', va='center', color=l.get_color(), size = 12)
        # g.annotate(f'{y[-9]:.1f}', xy=(0.19,y[-9]), xycoords=('axes fraction', 'data'), 
        #      ha='left', va='center', color=l.get_color(), size = 12)
ax.set_xlabel('Biennial Intervals',labelpad = 10, size = 16)
ax.set_ylabel('SEBS Participation Rates (%)',labelpad = 10, size = 16)
plt.legend(prop={'size': 14})
sns.despine()
plt.xticks(size = 12)
plt.yticks(size = 12)
ax.grid(True, color='grey', linestyle='--', linewidth=0.5, zorder=0)

# plt.title('Evolution du taux de participation au sein des clusters géographiques',size = 16)
# plt.savefig('../Manuscript/Figures - EN/Temporal trend Getis - Neighborhood.png', dpi = 300, bbox_inches = 'tight')

In [ ]:
fig, ax = plt.subplots(figsize = (15,8))
# the size of A4 paper
g = sns.lineplot(data=df_all_years_2020_neighborhood, x="year_invit", y="tx_screening", hue = 'tx_screening_G_cl',hue_order=
    ['Cold Spot - p < 0.001','Cold Spot - p < 0.01', 'Cold Spot - p < 0.05', 'Cold Spot - p < 0.1','Hot Spot - p < 0.001', 'Hot Spot - p < 0.01',
     'Hot Spot - p < 0.05', 'Hot Spot - p < 0.1', 'Not significant'], palette = colors_cl_getis,ax = ax)
for l in g.lines:
    y = l.get_ydata()
    if len(y)>0:
        g.annotate(f'{y[-1]:.1f}', xy=(0.97,y[-1]), xycoords=('axes fraction', 'data'), 
                     ha='left', va='center', color=l.get_color(), size = 12)
        # g.annotate(f'{y[-9]:.1f}', xy=(0.19,y[-9]), xycoords=('axes fraction', 'data'), 
        #      ha='left', va='center', color=l.get_color(), size = 12)
ax.set_xlabel('Biennial Intervals',labelpad = 10, size = 16)
ax.set_ylabel('Participation Rates (%)',labelpad = 10, size = 16)
plt.legend(prop={'size': 14})
sns.despine()
plt.xticks(size = 12)
plt.yticks(size = 12)
ax.grid(True, color='grey', linestyle='--', linewidth=0.5, zorder=0)

# plt.title('Evolution du taux de participation au sein des clusters géographiques',size = 16)
# plt.savefig('../Manuscript/Figures/Evolution temporelle Getis - Raw rates.png', dpi = 300, bbox_inches = 'tight')

In [ ]:
# gdf_neighborhood['ebs_rate_2000'] = gdf_neighborhood.nbid.map(dfs_year[0].set_index("nbid").ebs_rate.to_dict())
# gdf_neighborhood['ebs_rate_2002'] = gdf_neighborhood.nbid.map(dfs_year[1].set_index("nbid").ebs_rate.to_dict())
gdf_neighborhood['ebs_rate_2004'] = gdf_neighborhood.nbid.map(dfs_year[0].set_index("nbid").ebs_rate.to_dict())
gdf_neighborhood['ebs_rate_2006'] = gdf_neighborhood.nbid.map(dfs_year[1].set_index("nbid").ebs_rate.to_dict())
gdf_neighborhood['ebs_rate_2008'] = gdf_neighborhood.nbid.map(dfs_year[2].set_index("nbid").ebs_rate.to_dict())
gdf_neighborhood['ebs_rate_2010'] = gdf_neighborhood.nbid.map(dfs_year[3].set_index("nbid").ebs_rate.to_dict())
gdf_neighborhood['ebs_rate_2012'] = gdf_neighborhood.nbid.map(dfs_year[4].set_index("nbid").ebs_rate.to_dict())
gdf_neighborhood['ebs_rate_2014'] = gdf_neighborhood.nbid.map(dfs_year[5].set_index("nbid").ebs_rate.to_dict())
gdf_neighborhood['ebs_rate_2016'] = gdf_neighborhood.nbid.map(dfs_year[6].set_index("nbid").ebs_rate.to_dict())
gdf_neighborhood['ebs_rate_2018'] = gdf_neighborhood.nbid.map(dfs_year[7].set_index("nbid").ebs_rate.to_dict())
gdf_neighborhood['ebs_rate_2020'] = gdf_neighborhood.nbid.map(dfs_year[8].set_index("nbid").ebs_rate.to_dict())

# gdf_neighborhood['ebs_rate_2000'] = gdf_neighborhood.nbid.map(dfs_year[0].set_index("nbid").ebs_rate.to_dict())
# gdf_neighborhood['ebs_rate_2002'] = gdf_neighborhood.nbid.map(dfs_year[1].set_index("nbid").ebs_rate.to_dict())
gdf_neighborhood['raw_rate_2004'] = gdf_neighborhood.nbid.map(dfs_year[0].set_index("nbid").tx_screening.to_dict())
gdf_neighborhood['raw_rate_2006'] = gdf_neighborhood.nbid.map(dfs_year[1].set_index("nbid").tx_screening.to_dict())
gdf_neighborhood['raw_rate_2008'] = gdf_neighborhood.nbid.map(dfs_year[2].set_index("nbid").tx_screening.to_dict())
gdf_neighborhood['raw_rate_2010'] = gdf_neighborhood.nbid.map(dfs_year[3].set_index("nbid").tx_screening.to_dict())
gdf_neighborhood['raw_rate_2012'] = gdf_neighborhood.nbid.map(dfs_year[4].set_index("nbid").tx_screening.to_dict())
gdf_neighborhood['raw_rate_2014'] = gdf_neighborhood.nbid.map(dfs_year[5].set_index("nbid").tx_screening.to_dict())
gdf_neighborhood['raw_rate_2016'] = gdf_neighborhood.nbid.map(dfs_year[6].set_index("nbid").tx_screening.to_dict())
gdf_neighborhood['raw_rate_2018'] = gdf_neighborhood.nbid.map(dfs_year[7].set_index("nbid").tx_screening.to_dict())
gdf_neighborhood['raw_rate_2020'] = gdf_neighborhood.nbid.map(dfs_year[8].set_index("nbid").tx_screening.to_dict())

In [ ]:
nbid_center_density = gdf_participation.groupby('nbid').center_density.mean().to_dict()
nbid_center_nearest = gdf_participation.groupby('nbid').center_nearest.mean().to_dict()

## Space-time cube clustering

In [ ]:
import pymannkendall as mk
from itertools import groupby
from operator import itemgetter
import utils
from utils import translate_cat
from utils import masked_emergent_getis
import matplotlib.patches as mpatches
from matplotlib.path import Path
from matplotlib.patches import PathPatch

In [ ]:
df_space_time_cube = pd.concat(dfs_year)[['nbid','year_invit','ebs_rate_G_Zs','ebs_rate_G_cl', 'tx_screening_G_Zs', 'tx_screening_G_cl','geometry']]
df_space_time_cube['time'] = df_space_time_cube['year_invit'].astype(int).sub(2004).div(2)
# df_space_time_cube.groupby('RELI').ebs_rate_G_cl.apply(list)

In [ ]:
def get_consecutive(lst, max_time):
    consec_lsts = []
    for k, g in groupby(enumerate(lst), lambda ix : ix[0] - ix[1]):
        consecutive_list = list(map(itemgetter(1), g))
#         print(consecutive_list)
        consec_lsts.append(consecutive_list)
#         print(len(consec_lsts))
        if len(consec_lsts) > 1:
            return np.nan
        else:
            if max_time in consecutive_list and len(consecutive_list)>1:
                
                return consecutive_list
            else:
                return np.nan

In [ ]:
def get_consecutive_v2(lst, max_time):
    consec_lsts = []
    for k, g in groupby(enumerate(lst), lambda ix: ix[0] - ix[1]):
        consecutive_list = list(map(itemgetter(1), g))
        consec_lsts.append(consecutive_list)
    
    # Find sequences that end in max_time and have length > 1
    valid_sequences = [seq for seq in consec_lsts 
                      if max_time in seq and len(seq) > 1 and seq[-1] == max_time]
    
    # Should have exactly one valid sequence for "consecutive" pattern
    if len(valid_sequences) == 1:
        return valid_sequences[0]
    else:
        return np.nan

In [ ]:
def spatiotemporal_getis_cats_prep(df,class_col, z_col, max_time):
    df_cats_prep = pd.DataFrame(pd.DataFrame(df.nbid.unique(), columns = ['nbid'])).set_index('nbid')
    getis_cl_lst = df.groupby('nbid')[class_col].apply(list)
    getis_Zs_lst = df.groupby('nbid')[z_col].apply(list)
    getis_hs_index = getis_cl_lst.apply(lambda x: [i for i, s in enumerate(x) if 'Hot' in s])
    getis_cs_index = getis_cl_lst.apply(lambda x: [i for i, s in enumerate(x) if 'Cold' in s])
    df_cats_prep['n_periods'] = getis_cl_lst.apply(len)

    df_cats_prep['hs_index'] = getis_hs_index
    df_cats_prep['cs_index'] = getis_cs_index
    df_cats_prep['all_na_cnt'] = getis_cl_lst.apply(lambda x: sum(pd.isnull(s) for s in x))
    df_cats_prep['all_hs_cnt'] = getis_cl_lst.apply(lambda x: sum('Hot' in s for s in x))
    df_cats_prep['all_cs_cnt'] = getis_cl_lst.apply(lambda x: sum('Cold' in s for s in x))
    df_cats_prep['perc_hs'] = df_cats_prep['all_hs_cnt'].div(df['time'].nunique()).mul(100)
    df_cats_prep['perc_cs'] = df_cats_prep['all_cs_cnt'].div(df['time'].nunique()).mul(100)

    # Temporal trend
    df_cats_prep['trend'] = getis_Zs_lst.apply(lambda x: mk.original_test(x).trend if len(x) > 1 else np.nan)
    
    # Consecutive hot spot, consecutive cold spot
    df_cats_prep['consecutive_hs'] = getis_hs_index.apply(lambda x: get_consecutive_v2(x, max_time))
    df_cats_prep['consecutive_cs'] = getis_cs_index.apply(lambda x: get_consecutive_v2(x, max_time))
    df_cats_prep.loc[(df_cats_prep.hs_index.astype(str) == '[%s]'%(max_time)), 'class'] = 'Nouveau point chaud'
    df_cats_prep.loc[(df_cats_prep.perc_hs < 90)&(df_cats_prep.consecutive_hs.isnull()==False), 'class'] = 'Point chaud consécutif'
    df_cats_prep.loc[(df_cats_prep.perc_hs >= 90)&(df_cats_prep.consecutive_hs.isnull()==False)&(df_cats_prep.trend == 'increasing'), 'class'] = 'Intensification de point chaud'
    df_cats_prep.loc[(df_cats_prep.perc_hs >= 90)&(df_cats_prep.consecutive_hs.isnull()==False)&(df_cats_prep.trend == 'no trend'), 'class'] = 'Point chaud persistant'
    df_cats_prep.loc[(df_cats_prep.perc_hs >= 90)&(df_cats_prep.consecutive_hs.isnull()==False)&(df_cats_prep.trend == 'decreasing'), 'class'] = 'Point chaud diminuant'
    df_cats_prep.loc[(df_cats_prep.perc_hs < 90)&(df_cats_prep.hs_index.astype(str).str.contains('%s]'%(max_time)))&(df_cats_prep.all_cs_cnt == 0)&(df_cats_prep.all_hs_cnt > 1), 'class'] = 'Point chaud sporadique'
    df_cats_prep.loc[(df_cats_prep.perc_hs < 90)&(df_cats_prep.hs_index.astype(str).str.contains('%s]'%(max_time)))&(df_cats_prep.all_cs_cnt > 0), 'class'] = 'Point chaud oscillant'
    df_cats_prep.loc[(df_cats_prep.perc_hs >= 50)&(df_cats_prep.hs_index.astype(str).str.contains('%s]'%(max_time)) == False), 'class'] = 'Point chaud historique'

    df_cats_prep.loc[(df_cats_prep.cs_index.astype(str) == '[%s]'%(max_time)), 'class'] = 'Nouveau point froid'
    df_cats_prep.loc[(df_cats_prep.perc_cs < 90)&(df_cats_prep.consecutive_cs.isnull()==False), 'class'] = 'Point froid consécutif'
    df_cats_prep.loc[(df_cats_prep.perc_cs >= 90)&(df_cats_prep.consecutive_cs.isnull()==False)&(df_cats_prep.trend == 'decreasing'), 'class'] = 'Intensification de point froid'
    df_cats_prep.loc[(df_cats_prep.perc_cs >= 90)&(df_cats_prep.consecutive_cs.isnull()==False)&(df_cats_prep.trend == 'no trend'), 'class'] = 'Point froid persistant'
    df_cats_prep.loc[(df_cats_prep.perc_cs >= 90)&(df_cats_prep.consecutive_cs.isnull()==False)&(df_cats_prep.trend == 'increasing'), 'class'] = 'Point froid diminuant'
    df_cats_prep.loc[(df_cats_prep.perc_cs < 90)&(df_cats_prep.cs_index.astype(str).str.contains('%s]'%(max_time)))&(df_cats_prep.all_hs_cnt == 0)&(df_cats_prep.all_cs_cnt > 1), 'class'] = 'Point froid sporadique'
    df_cats_prep.loc[(df_cats_prep.perc_cs < 90)&(df_cats_prep.cs_index.astype(str).str.contains('%s]'%(max_time)))&(df_cats_prep.all_cs_cnt > 0), 'class'] = 'Point froid oscillant'
    df_cats_prep.loc[(df_cats_prep.perc_cs >= 50)&(df_cats_prep.cs_index.astype(str).str.contains('%s]'%(max_time)) == False), 'class'] = 'Point froid historique'
    df_cats_prep.loc[df_cats_prep['class'].isnull(),'class'] = 'Aucun modèle détecté'
    return df_cats_prep

In [ ]:
df_emergent_getis = spatiotemporal_getis_cats_prep(df_space_time_cube,'ebs_rate_G_cl','ebs_rate_G_Zs', 8)

In [ ]:
df_emergent_getis_test = spatiotemporal_getis_cats_prep(df_space_time_cube,'tx_screening_G_cl','tx_screening_G_Zs', 8)

In [ ]:
df_emergent_getis['class'] = translate_cat(df_emergent_getis['class'],'FR')
df_emergent_getis_test['class'] = translate_cat(df_emergent_getis_test['class'],'FR')

In [ ]:
df_emergent_getis_n9 = df_emergent_getis[df_emergent_getis.n_periods == 9] # Restricted to neighborhoods present all periods
df_emergent_getis_test_n9 = df_emergent_getis_test[df_emergent_getis_test.n_periods == 9] # Restricted to neighborhoods present all periods

In [ ]:
df_emergent_getis['geometry'] = gdf_neighborhood.set_index('nbid')['geometry']
df_emergent_getis_n9['geometry'] = gdf_neighborhood.set_index('nbid')['geometry']

df_emergent_getis_test['geometry'] = gdf_neighborhood.set_index('nbid')['geometry']
df_emergent_getis_test_n9['geometry'] = gdf_neighborhood.set_index('nbid')['geometry']

In [ ]:
df_emergent_getis = gpd.GeoDataFrame(df_emergent_getis, crs = 2056, geometry = df_emergent_getis['geometry'])
df_emergent_getis_n9 = gpd.GeoDataFrame(df_emergent_getis_n9, crs = 2056, geometry = df_emergent_getis_n9['geometry'])

df_emergent_getis_test = gpd.GeoDataFrame(df_emergent_getis_test, crs = 2056, geometry = df_emergent_getis_test['geometry'])
df_emergent_getis_test_n9 = gpd.GeoDataFrame(df_emergent_getis_test_n9, crs = 2056, geometry = df_emergent_getis_test_n9['geometry'])

In [ ]:
colors_cl_getis = { 'Nouveau point froid':'#eff3ff',
                    'Point froid diminuant': "#80cdc1",
                    'Point froid persistant':'#313695',
                    'Point froid historique':'#2166ac',
                    'Point froid sporadique':'#67a9cf',
                    'Point froid oscillant':'#d1e5f0',
                    'Intensification de point froid':'#036ffc',
                    'Point chaud persistant':'#e41a1c',
                    'Point chaud historique':'#b2182b',
                    'Point chaud sporadique':'#ef8a62',
                    'Point chaud oscillant':'#fddbc7',
                    'Intensification de point chaud':'#e7298a',
                    'Nouveau point chaud':'#ffeda0',
                    'Point chaud diminuant': "#fb9a99",
                    'Aucun modèle détecté':'#f0f0f0'}

colors_cl_getis_en = {
# 'New cold spot': '#eff3ff',
'Diminishing cold spot': "#80cdc1",
'Persistent cold spot': '#313695',
'Historical cold spot': '#2166ac',
# 'Sporadic cold spot': '#67a9cf',
'Oscillating cold spot': '#d1e5f0',
'Intensifying cold spot': '#036ffc',
'Persistent hot spot': '#e41a1c',
'Historical hot spot': '#b2182b',
'Sporadic hot spot': '#ef8a62',
'Oscillating hot spot': '#fddbc7',
'Intensifying hot spot': '#e7298a',
'New hot spot': '#ffeda0',
'Diminishing hot spot': "#fb9a99",
'No pattern detected': '#f0f0f0'
}
hmap = colors.ListedColormap([colors_cl_getis_en[i] for i in df_emergent_getis['class'].sort_values().unique()])
# hmap = colors.ListedColormap([colors_cl_getis_en[i] for i in df_emergent_getis_test['class'].sort_values().unique()])

### Distribution of # of periods of presence

In [ ]:
n_by_n_periods = pd.DataFrame(df_emergent_getis.groupby('n_periods').size(), columns = ['n']).reset_index()
perc_by_n_periods = pd.DataFrame(df_emergent_getis.groupby('n_periods').size(), columns = ['n']).reset_index()
perc_by_n_periods['n'] = perc_by_n_periods['n']/21.12

In [ ]:
# Create the figure and the axes
fig, ax = plt.subplots(figsize=(6, 6))
sns.barplot(x='n_periods', y='n', data=n_by_n_periods, color = 'grey', ax=ax)
# Add title and labels
# plt.title('Nombre de Patients par Assureur', fontsize=16)
plt.xlabel('Number of represented time intervals', fontsize=12)
plt.ylabel('Number of Swiss Neighborhoods', fontsize=12)
# Despine visual elements for a cleaner look
sns.despine()
ax.grid(True, color='grey', linestyle='--', linewidth=0.5, zorder=0)
# plt.savefig('../Manuscript/Figures - EN/Distribution of represented periods - Neighborhoods.png', dpi = 300, bbox_inches = 'tight')

In [ ]:
# Create the figure and the axes
fig, ax = plt.subplots(figsize=(6,6))
sns.barplot(x='n_periods', y='n', data=perc_by_n_periods, color = 'grey', ax=ax)
# Add title and labels
# plt.title('Nombre de Patients par Assureur', fontsize=16)
plt.xlabel('Number of represented time intervals', fontsize=12)
plt.ylabel('Percentage of Swiss Neighborhoods', fontsize=12)
# Despine visual elements for a cleaner look
sns.despine()
ax.grid(True, color='grey', linestyle='--', linewidth=0.5, zorder=0)
# plt.savefig('../Manuscript/Figures - EN/Distribution of represented periods (%) - Neighborhoods.png', dpi = 300, bbox_inches = 'tight')

In [ ]:
# Person-neighborhood observations (corrected to exclude 2000, 2002)

n_women_by_nbid = gdf_participation[gdf_participation.year_invit_2y.isin(['2000','2002'])==False].groupby('nbid')['numerodossier'].nunique()
n_mammo_by_nbid = gdf_participation[gdf_participation.year_invit_2y.isin(['2000','2002'])==False].groupby('nbid')['mammo'].sum()

df_emergent_getis['n_mammo'] = df_emergent_getis.index.map(n_mammo_by_nbid)
df_emergent_getis['n_women_by_nbid'] = df_emergent_getis.index.map(n_women_by_nbid)


samples = df_emergent_getis.groupby(['class']).n_women_by_nbid.sum().astype(str)
total_samples = df_emergent_getis.n_women_by_nbid.sum()
percentages = (df_emergent_getis.groupby('class').n_women_by_nbid.sum()/ total_samples).mul(100).round(1).astype(str)

In [ ]:
samples_formatted = samples.astype(int).apply(lambda x: f'{x:,}')


In [ ]:
# Ensure 'samples' is a Series or something that can be used to sort 'df_emergent_getis'
df_emergent_getis['samples_count'] = df_emergent_getis['class'].map(samples)

# Sort the DataFrame based on the count of samples
df_emergent_getis_sorted = df_emergent_getis.sort_values(by='samples_count', ascending=False)

In [ ]:
samples_dict = samples.to_dict()
# samples_dict['New cold spot'] = '0'
# samples_dict['Sporadic cold spot'] = '0'

In [ ]:
# # y = df_emergent_getis_sorted['class'] + ' - ' +  df_emergent_getis_sorted['class'].map(samples) + ' ('+ df_emergent_getis_sorted['class'].map(percentages) + '%)'
# category_name = df_emergent_getis_sorted['class'].astype(str)
# n_by_category = df_emergent_getis_sorted['class'].map(samples).astype(int)  # Keep as int first
# n_by_category_formatted = n_by_category.apply(lambda x: f'{x:,}')  # Then format
# perc_by_category = df_emergent_getis_sorted['class'].map(percentages).astype(str)

# y = (category_name + ' - ' + 
#      n_by_category_formatted + 
#      ' (' + perc_by_category + '%)')

# df_emergent_getis_sorted['plot_labels'] = y

ax = df_emergent_getis_sorted.plot('class', cmap = hmap, figsize = (12,12), zorder = 3, linewidth = 0.01, legend = False)
cantons[cantons.NAME == 'Genève'].geometry.boundary.plot(color = 'lightgrey', facecolor='None', ax = ax, zorder = 1)
lake.plot(color = 'lightblue',ax = ax, zorder = 4)
ax.set_axis_off()
# communes.apply(lambda x: ax.annotate(text=x.NAME, xy=x.geometry.centroid.coords[0], ha='center', size=6, alpha = 0.8), axis=1)
lake.apply(lambda x: ax.annotate(text=x.NOM, xy=x.geometry.centroid.coords[0], ha='center', size=12, alpha = 0.8, zorder=5), axis=1)


# Add Geneva city boundary and annotation
geneva_city = communes[communes.NAME == 'Genève']
geneva_city.boundary.plot(ax=ax, color='black', linewidth=0.8, zorder = 3)

communes.boundary.plot(ax=ax, color='black', linewidth=0.2, zorder = 3)

# Add Geneva city label at centroid
centroid = geneva_city.geometry.centroid.iloc[0]
ax.annotate('Geneva', xy=(centroid.x, centroid.y), fontsize=12, fontweight='bold', 
            ha='center', va='center', zorder = 5)


# legend = ax.get_legend()
# # Change the fontsize
# legend.get_title().set_fontsize(12)

gdf_centre.plot(c = '#31a354', markersize = 40, alpha = 1, marker = 'x', ax = ax, zorder = 6)

# Reorder legend by sample count (N)
legend_data = []
for category, color in colors_cl_getis_en.items():
    count = samples_dict[category]  # Get N for this category
    count = int(count)
    legend_data.append((count, category, color))

# Sort by count (descending)
legend_data.sort(key=lambda x: x[0], reverse=True)

# Create ordered legend elements
legend_elements = []
for count, category, color in legend_data:
    legend_elements.append(patches.Patch(color=color, label=f'{category} - {count:,} ({count/total_samples*100:.1f}%)'))

# Add screening centers to legend
legend_elements.append(plt.Line2D([0], [0], marker='x', color='#31a354', linestyle='None', 
                                 markersize=10, markeredgewidth=3, label='Breast screening centers'))

# Apply the ordered legend
ax.legend(handles=legend_elements, loc='upper left', 
          fontsize=10, title='Categories - N women (%)', title_fontsize=11)
scalebar = ScaleBar(1, units="m", location="lower right")
ax.add_artist(scalebar)
plt.savefig('../Manuscript/Figures - EN/Emergent Getis - Neighborhood - REVIEW.png', transparent=True, dpi = 400, bbox_inches = 'tight')
plt.show()

In [ ]:
df_emergent_getis_sorted['labels'] = df_emergent_getis_sorted['class'] + ' - ' +  df_emergent_getis_sorted['class'].map(samples_formatted) + ' ('+ df_emergent_getis_sorted['class'].map(percentages) + '%)'
dict_labels = df_emergent_getis_sorted.set_index('class')['labels'].to_dict()

In [ ]:
for category in df_emergent_getis['class'].unique():
    print(category)
    utils.masked_emergent_getis(df_emergent_getis, gdf_centre, category, colors_cl_getis_en, dict_labels)
    plt.savefig('../Manuscript/Figures - EN/Biennial_2004_2020_emergent_Getis_%s - Neighborhoods.png'%(category), dpi = 300, bbox_inches = 'tight')

In [ ]:
df_emergent_getis_n9['n_mammo'] = df_emergent_getis_n9.index.map(n_mammo_by_nbid)
df_emergent_getis_n9['n_women_by_nbid'] = df_emergent_getis_n9.index.map(n_women_by_nbid)


samples = df_emergent_getis_n9.groupby(['class']).n_women_by_nbid.sum().astype(str)
total_samples = df_emergent_getis_n9.n_women_by_nbid.sum()
percentages = (df_emergent_getis_n9.groupby('class').n_women_by_nbid.sum()/ total_samples).mul(100).round(1).astype(str)
samples_dict = samples.to_dict()


In [ ]:
gpd.sjoin(gdf_participation[gdf_participation.year_invit_2y.isin(['2000','2002'])==False], df_emergent_getis_n9, predicate='within').numerodossier.nunique()

In [ ]:
y = df_emergent_getis_n9['class'] + ' - ' +  df_emergent_getis_n9['class'].map(samples) + ' ('+ df_emergent_getis_n9['class'].map(percentages) + '%)'
ax = df_emergent_getis_n9.plot(y, cmap = hmap, figsize = (12,12), zorder = 3, linewidth = 0.01, legend = True, legend_kwds = {'title':'Categories - N women (%)','fontsize':10, 'loc':'upper left'})
# communes.plot(color = 'lightgrey', ax = ax, zorder = 1)
lake.plot(color = 'lightblue',ax = ax, zorder = 4)
ax.set_axis_off()
# communes.apply(lambda x: ax.annotate(text=x.NAME, xy=x.geometry.centroid.coords[0], ha='center', size=6, alpha = 0.8), axis=1)
lake.apply(lambda x: ax.annotate(text=x.NOM, xy=x.geometry.centroid.coords[0], ha='center', size=12, alpha = 0.8), axis=1)

# Add Geneva city boundary and annotation
geneva_city = communes[communes.NAME == 'Genève']
geneva_city.boundary.plot(ax=ax, color='black', linewidth=0.8, zorder = 3)

communes.boundary.plot(ax=ax, color='black', linewidth=0.2, zorder = 3)

# Add Geneva city label at centroid
centroid = geneva_city.geometry.centroid.iloc[0]
ax.annotate('Geneva', xy=(centroid.x, centroid.y), fontsize=12, fontweight='bold', 
            ha='center', va='center', zorder = 5)



gdf_centre.plot(c = '#31a354', markersize = 30, alpha = 1, marker = 'x', ax = ax, zorder = 6)


# Reorder legend by sample count (N)
legend_data = []
for category, color in colors_cl_getis_en.items():
    count = samples_dict[category]  # Get N for this category
    count = int(count)
    legend_data.append((count, category, color))

# Sort by count (descending)
legend_data.sort(key=lambda x: x[0], reverse=True)

# Create ordered legend elements
legend_elements = []
for count, category, color in legend_data:
    legend_elements.append(patches.Patch(color=color, label=f'{category} - {count:,} ({count/total_samples*100:.1f}%)'))

# Add screening centers to legend
legend_elements.append(plt.Line2D([0], [0], marker='x', color='#31a354', linestyle='None', 
                                 markersize=10, markeredgewidth=3, label='Breast screening centers'))

# Apply the ordered legend
ax.legend(handles=legend_elements, loc='upper left', 
          fontsize=10, title='Categories - N women (%)', title_fontsize=11)
add_scale_bar(ax)
plt.savefig('../Manuscript/Figures - EN/Emergent Getis - 9 periods only - Neighborhood.png', dpi = 300, bbox_inches = 'tight')

### Descriptive analyses by SPTM class

In [ ]:
df_all_years_2020_neighborhood = df_all_years_2020_neighborhood.set_index('nbid')
df_all_years_2020_neighborhood['class_emergent_getis'] = df_emergent_getis['class']

In [ ]:
data_2020 = df_all_years_2020_neighborhood[df_all_years_2020_neighborhood.year_invit == '2020']

In [ ]:
data_2020['class_emergent_getis'] = data_2020['class_emergent_getis'].astype('string')

In [ ]:
sns.set_context("paper")
flier_props = {'marker':'D', 'markersize':2}

In [ ]:
def plot_participation_by_class(data, y_column, x_column, annotator_pairs, plot_type, output_file, colors_cl_getis_en, context="paper", figsize=(6, 6), rotation=45, size=8):
    from statannotations.Annotator import Annotator

    sns.set_context(context)
    fig, ax = plt.subplots(figsize=figsize)
    colors_emergentGetis = [colors_cl_getis_en[i] for i in data[x_column].sort_values().unique()]
    ylim_max = data[y_column].quantile(q=0.99)
    x_label = 'Emerging hot spot categories'
    if plot_type == 'barplot':
        chart = sns.barplot(x=x_column, y=y_column, order = list([i for i in data[x_column].sort_values().unique()]),
                        palette=colors_emergentGetis, data=data, ax=ax)
    if plot_type == 'boxplot':
        chart = sns.boxplot(x=x_column, y=y_column,order = list([i for i in data[x_column].sort_values().unique()]),
                    palette=colors_emergentGetis, showfliers=True, flierprops=flier_props, data=data, ax=ax)
    chart.set_xticklabels(chart.get_xticklabels(), size=size, rotation=rotation, horizontalalignment='right')
    chart.set_xlabel(x_label, size=12)
    chart.set_ylabel('SEBS Participation Rate (%)', size=12)

    # plt.savefig(output_file, dpi = 300, bbox_inches = 'tight')

In [ ]:
plot_participation_by_class(data_2020, 'ebs_rate', 'class_emergent_getis', [('No pattern detected',"Persistent cold spot"), ('No pattern detected','Persistent hot spot')],'boxplot', '../Manuscript/Figures - EN/Boxplot - Emergent Getis - Tx Participation - Neighborhood.png', colors_cl_getis_en)

In [ ]:
df_all_years_2020_neighborhood.groupby(['year_invit','class_emergent_getis'])['ebs_rate'].mean()

In [ ]:
fig, ax = plt.subplots(figsize = (8, 8))
# the size of A4 paper
g = sns.lineplot(data=df_all_years_2020_neighborhood[df_all_years_2020_neighborhood.year_invit.isin(['2000','2002'])==False].reset_index(), x="year_invit", y="ebs_rate", hue = 'class_emergent_getis', linewidth = 2, palette = colors_cl_getis_en,ax = ax)
for l in g.lines:
    y = l.get_ydata()
    if len(y)>0:
        g.annotate(f'{y[-1]:.1f}', xy=(1.02,y[-1]), xycoords=('axes fraction', 'data'), 
                     ha='right', va='center', color=l.get_color(), size = 10)
ax.set_xlabel('Biennial interval',labelpad = 10, size = 16)
ax.set_ylabel('SEBS Participation Rate (%)',labelpad = 10, size = 16)
plt.legend(prop={'size': 16})
plt.xticks(size = 12)
plt.yticks(size = 12)
ax.grid(True, color='grey', linestyle='--', linewidth=0.5, zorder=0)

# get the legend object
leg = ax.legend()
sns.despine()
# change the line width for the legend
for line in leg.get_lines():
    line.set_linewidth(4.0)
# plt.title('Evolution du taux de participation au sein des différentes catégories de clusters',size = 16)
# plt.savefig('../Manuscript/Figures - EN/Emergent Getis - Evolution au cours du temps par classe - Neighborhood.png', dpi = 300, bbox_inches = 'tight')
plt.show()

In [ ]:
data_2020 = data_2020.reset_index()

In [ ]:
data_2020['center_density'] = data_2020.nbid.map(nbid_center_density)
data_2020['center_nearest'] = data_2020.nbid.map(nbid_center_nearest)

data_2020 = pd.merge(data_2020, microgis_data.drop('geometry',axis=1), how='left', on = 'nbid')

In [ ]:
columns = ['nbid', '30-34', '35-39', '40-44', '45-49', '50-54', 
    '55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85-89', '90-94']

# Now perform the selection and grouping
_df_by_nbid = gdf_participation[columns].groupby('nbid').mean().astype(float).mul(100).reset_index()

In [ ]:
data_2020 = pd.merge(data_2020, _df_by_nbid, how='left', on = 'nbid')

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# Scale the column
data_2020['deprivation_pca_scaled'] = scaler.fit_transform(-data_2020[['deprivation_pca']])

In [ ]:
colors_emergentGetis = [colors_cl_getis_en[i] for i in data_2020['class_emergent_getis'].sort_values().unique()]
ylim_max = data_2020['ebs_rate'].quantile(q=0.99)
x_label = ''
# y_label = 'Taux de participation (%) en 2020'
features = ['deprivation_pca_scaled','center_density']
y_labels = ['SES deprivation index (scaled)','Access - Screening center density']

In [ ]:
data_2020.shape

In [ ]:
for feature, y_label in zip(features, y_labels):
    fig, ax = plt.subplots(figsize=(4, 4))
#     f = plt.figure(figsize = (5,5))
    chart = sns.boxplot(x='class_emergent_getis', y=feature,order = list([i for i in data_2020['class_emergent_getis'].sort_values().unique()]),
                        palette=colors_emergentGetis, data=data_2020, flierprops = flier_props, ax=ax)
    chart.set_xticklabels(chart.get_xticklabels(), size=8, rotation=45, horizontalalignment='right')
    chart.set_title('{}'.format(y_label, size=16))
    chart.set_xlabel(x_label, size=10)
    chart.set_ylabel(y_label, size=10)
    plt.tight_layout()
    # plt.savefig(f'../Manuscript/Figures - EN/Box plot - Emergent Getis_{y_label} - Neighborhood.png', dpi = 300, bbox_inches = 'tight')

## Modelling 

In [ ]:
from sklearn.model_selection import train_test_split
import xgboost
import shap
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
data_2020[['deprivation_pca','deprivation_pca_scaled']]

In [ ]:
old_names = ['ciqmd','center_density','center_nearest','deprivation_pca_scaled','dmdrent','wo_tertiary_education','rpnch','radune',"ptot",'rad3tert',
 '50-54',
 '55-59',
 '60-64',
 '65-69',
 '70-74',]

new_names = ['Median income (CHF)','Center density','Nearest center','SES deprivation','Median monthly rent (CHF)','Tertiary education (%)','Swiss nationals (%)','Unemployment (%)',"Population count",'Tertiary sector (%)',
 '50-54',
 '55-59',
 '60-64',
 '65-69',
 '70-74']

In [ ]:
dict_col_names = {k:v for k,v in zip(old_names,new_names)}

In [ ]:
dict_col_names['e'] = 'X-coord'
dict_col_names['n'] = 'Y-coord'
dict_col_names['ebs_rate'] = 'SEBS participation rate (%)'
dict_col_names['tx_screening'] = 'Participation rate (%)'

In [ ]:
df_modelling = data_2020.copy()
df_modelling.name = 'Participation'

In [ ]:
df_modelling['e'] = df_modelling.geometry.centroid.x
df_modelling['n'] = df_modelling.geometry.centroid.y

In [ ]:
df_modelling = df_modelling.rename(columns=dict_col_names)

In [ ]:
%%time
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval

# Fixed hyperparameter space
space = {
    'max_depth': hp.choice('max_depth', np.arange(3, 15, 1, dtype=int)),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.3, 1.0, 0.05),
    'min_child_weight': hp.choice('min_child_weight', np.arange(1, 10, 1, dtype=int)),
    'subsample': hp.quniform('subsample', 0.5, 1.0, 0.05),
    'learning_rate': hp.quniform('learning_rate', 0.01, 0.3, 0.01),  # More realistic range
    'gamma': hp.quniform('gamma', 0.0, 2.0, 0.1),
    'reg_alpha': hp.quniform('reg_alpha', 0.0, 1.0, 0.1),    # L1 regularization
    'reg_lambda': hp.quniform('reg_lambda', 1.0, 2.0, 0.1),  # L2 regularization
    
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
}

def score(params, X_train, y_train, n_folds=5):  # Pass data explicitly
    """Cross-validation scoring function"""
    d_train = xgboost.DMatrix(X_train, y_train)
    
    cv_results = xgboost.cv(
        params, d_train, 
        nfold=n_folds, 
        num_boost_round=500,
        early_stopping_rounds=10, 
        metrics='rmse', 
        seed=0,
        verbose_eval=False  # Reduce output
    )
    
    loss = cv_results['test-rmse-mean'].min()
    return loss

def optimize(trials, space, n_evals, X_train, y_train):
    """Optimization function with data passed explicitly"""
    
    # Create a closure to pass data to score function
    def objective(params):
        return score(params, X_train, y_train)
    
    best = fmin(
        objective, 
        space, 
        algo=tpe.suggest, 
        max_evals=n_evals,
        trials=trials,
        rstate=np.random.default_rng(333)
    )
    return best

In [ ]:
def interpretable_ml_shap_viz(df, y_col, final_model, X_coords, explainer_shap, shap_values, model_directory, include_spatial):
    import warnings
    # Suppress NumPy RNG warnings from SHAP
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=FutureWarning, message=".*NumPy global RNG.*")
        
        if not os.path.exists(model_directory):
            os.makedirs(model_directory)
        
        # Shapley summary
        shap.summary_plot(shap_values, feature_names=X_coords.columns, show=False)
        plt.savefig(model_directory / 'shap_summary.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # Shapley interaction values
        shap_interaction_values = explainer_shap.shap_interaction_values(X_coords)
        shap.summary_plot(shap_interaction_values, X_coords, max_display=16,
                          feature_names=X_coords.columns, show=False, plot_type="compact_dot")
        plt.savefig(model_directory / 'shap_summary_interaction_values.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # Shapley dependence plots
        dependence_dir = model_directory / 'Dependence plots'
        if not os.path.exists(dependence_dir):
            os.makedirs(dependence_dir)
        
        for name in X_coords.columns:
            shap.dependence_plot(name, shap_values.values, X_coords, display_features=X_coords, show=False)
            plt.savefig(dependence_dir / f'shap_dependence_{name}.png', dpi=300, bbox_inches='tight')
            plt.close()

    # Maps of SHAP variables (spatial visualization)
    if include_spatial:
        n_columns = len(X_coords.columns)
        n_rows = int(np.ceil(n_columns / 3))
        
        fig, ax = plt.subplots(n_rows, 3, figsize=(15, 15 * n_rows / 3))
        ax = ax.ravel()

        for j in range(n_columns):
            df.plot(ax=ax[j], column=shap_values.values[:, j], legend=True,
                    vmin=-0.8, vmax=0.8, cmap=shap.plots.colors.red_white_blue, legend_kwds={'shrink':0.5})
            ax[j].set_title("SHAP for\n" + str(X_coords.columns[j]), fontsize=10)
            ax[j].set_axis_off()

        # Hide remaining unused subplots
        for j in range(n_columns, n_rows * 3):
            ax[j].set_axis_off()

        plt.tight_layout()
        plt.savefig(model_directory / 'maps_shap_variables.png', dpi=300, bbox_inches='tight')
        plt.close()

        # Map of location effects (assuming last 2 columns are X-coord, Y-coord)
        spatial_shap = shap_values.values[:, -2:].sum(axis=1)
        fig, ax = plt.subplots(dpi=300)
        df.plot(ax=ax, column=spatial_shap, legend=True, vmin=-0.6, vmax=0.6, 
                figsize=(15, 8), cmap=shap.plots.colors.red_white_blue)
        plt.title("Location Effect on Predictions\n(SHAP values of geographic coordinates)\n", fontsize=8)
        plt.axis('off')
        plt.savefig(model_directory / 'maps_shap_location.png', dpi=300, bbox_inches='tight')
        plt.close()

    # Variable importance
    xgboost.plot_importance(final_model)
    plt.title("Feature Importance (Gain)")
    plt.savefig(model_directory / 'importance_plot_default.png', dpi=300, bbox_inches='tight')
    plt.close()

    xgboost.plot_importance(final_model, importance_type="cover")
    plt.title('Feature Importance (Cover)')
    plt.savefig(model_directory / 'importance_plot_cover.png', dpi=300, bbox_inches='tight')
    plt.close()

    # Force plot
    shap.force_plot(explainer_shap.expected_value, shap_values.values[1, :], X_coords.iloc[1, :], 
                    show=False, matplotlib=True)
    plt.savefig(model_directory / 'force_plot.png', dpi=300, bbox_inches='tight')
    plt.close()
        
    return df


def train_xgboost(gwr_df, X_eq, y_col, optimize, space, n_evals, file_prefix, include_spatial=True, plot_figures=True):
    df_model = gwr_df.copy(deep=True)
    model_directory = result_folder / 'Modelling_REVIEW' / file_prefix
    if not os.path.exists(model_directory):
        os.makedirs(model_directory)
    
    # Prepare features
    if include_spatial:
        X_coords = df_model[X_eq + ['X-coord', 'Y-coord']]
    else:
        X_coords = df_model[X_eq]
    
    y = df_model[y_col]
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_coords, y, test_size=0.2, random_state=0)
    
    # Prepare DMatrix objects
    d_train = xgboost.DMatrix(X_train, label=y_train)
    d_test = xgboost.DMatrix(X_test, label=y_test)
    d_all = xgboost.DMatrix(X_coords, label=y)
    
    ### Hyperparameter tuning
    if not os.path.exists(model_directory / 'best_params.pkl'):
        trials = Trials()
        best_params = optimize(trials, space, n_evals=n_evals, X_train=X_train, y_train=y_train)
        best_params = space_eval(space, best_params)
        
        with open(model_directory / 'trials.pkl', 'wb') as f:
            pickle.dump(trials, f)
        with open(model_directory / 'best_params.pkl', 'wb') as f:
            pickle.dump(best_params, f)
        
        print(f"Hyperparameter optimization completed for {file_prefix}")
    else:
        with open(model_directory / 'best_params.pkl', 'rb') as f:
            best_params = pickle.load(f)
        print(f"Loaded existing hyperparameters for {file_prefix}")

    ### Train model (single training approach)
    if not os.path.exists(model_directory / 'model.pkl'):
        final_model = xgboost.train(
            best_params, 
            d_train, 
            num_boost_round=5000, 
            evals=[(d_train, "train"), (d_test, "test")], 
            verbose_eval=False, 
            early_stopping_rounds=10
        )
        with open(model_directory / 'model.pkl', 'wb') as f:
            pickle.dump(final_model, f)
    else:
        with open(model_directory / 'model.pkl', 'rb') as f:
            final_model = pickle.load(f)

    ### Evaluation - separate test and full dataset
    # Test set evaluation (unbiased performance)
    y_pred_test = final_model.predict(d_test)
    print("Test RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_test)))
    print("Test R2:", r2_score(y_test, y_pred_test))
    
    # Full dataset predictions (for visualization and SHAP)
    y_pred_all = final_model.predict(d_all)
    print('Full dataset RMSE:', np.sqrt(mean_squared_error(y, y_pred_all)))
    print('Full dataset R2:', r2_score(y, y_pred_all))
    
    ### SHAP Analysis
    explainer_shap = shap.TreeExplainer(final_model)
    shap_values = explainer_shap(X_coords)
    
    # Debug SHAP calculation
    print(f"SHAP base_values shape: {np.array(shap_values.base_values).shape}")
    print(f"SHAP values shape: {shap_values.values.shape}")
    print(f"Expected predictions shape: {y_pred_all.shape}")
    
    # Correct SHAP interpretation - handle scalar vs array base_values
    if np.isscalar(shap_values.base_values):
        # Single base value for all predictions
        shap_predictions = shap_values.base_values + shap_values.values.sum(axis=1)
    else:
        # Array of base values (one per prediction)
        shap_predictions = shap_values.base_values + shap_values.values.sum(axis=1)
    
    # Alternative method: use shap_values.data if available
    if hasattr(shap_values, 'data') and shap_values.data is not None:
        # Direct prediction from SHAP explanation
        shap_predictions_alt = explainer_shap.expected_value + shap_values.values.sum(axis=1)
        print(f"Using explainer.expected_value: {explainer_shap.expected_value}")
        shap_predictions = shap_predictions_alt
    
    # Add results to dataframe
    df_model['predictions'] = y_pred_all
    df_model['shap_predictions'] = shap_predictions
    df_model['residuals'] = df_model[y_col] - df_model['predictions']
    
    # Verify SHAP calculations with detailed debugging
    shap_diff = np.abs(df_model['predictions'] - df_model['shap_predictions'])
    max_diff = shap_diff.max()
    mean_diff = shap_diff.mean()
    
    print(f"SHAP vs XGB prediction differences:")
    print(f"  Max difference: {max_diff:.6f}")
    print(f"  Mean difference: {mean_diff:.6f}")
    print(f"  Std difference: {shap_diff.std():.6f}")
    
    if max_diff > 1e-3:  # More lenient threshold
        print(f"Warning: SHAP predictions have notable differences from XGB predictions")
        print("This might be normal for complex models with interactions")
    else:
        print("✓ SHAP predictions match XGB predictions within tolerance")
    
    ### Generate visualizations
    if plot_figures:
        df_model = interpretable_ml_shap_viz(df_model, y_col, final_model, X_coords, 
                                           explainer_shap, shap_values, model_directory, include_spatial)

    return final_model, df_model, shap_values

In [ ]:
# model_directory = result_folder / 'Modelling' / 'Model 0'
# if not os.path.exists(model_directory):
#     os.makedirs(model_directory)

In [ ]:
y_col = 'Participation rate (%)'  # Keep this name for column reference
X_eq0 = ['Median income (CHF)','Center density','Median monthly rent (CHF)','Tertiary education (%)','Unemployment (%)','Tertiary sector (%)','Swiss nationals (%)',"Population count",
 '50-54','55-59','60-64','65-69','70-74']
X_eq1 = ['SES deprivation','Center density', '50-54','55-59','60-64','65-69','70-74']
X_eq2 = ['SES deprivation']
X_eq3 = ['Center density']
X_eq4 = ['Nearest center']

include_spatial = True
n_evals = 200
df_model = df_modelling.copy(deep=True)

In [ ]:
X_eq0 = ['Median income (CHF)','Center density','Median monthly rent (CHF)',
         'Tertiary education (%)','Unemployment (%)','Tertiary sector (%)',
         'Swiss nationals (%)',"Population count",
         '50-54','55-59','60-64','65-69','70-74']

X_eq1 = ['SES deprivation','Center density', 
         '50-54','55-59','60-64','65-69','70-74']

X_eq2 = ['SES deprivation']

X_eq3 = ['Center density']

X_eq4 = ['Nearest center']

# Common parameters
y_col = 'Participation rate (%)'
n_evals = 200

# Verify that spatial coordinates exist in the dataframe
required_coords = ['X-coord', 'Y-coord']
if not all(coord in df_model.columns for coord in required_coords):
    raise ValueError(f"Missing spatial coordinates. Required: {required_coords}")

# Verify that all required features exist
all_features = set(X_eq0 + X_eq1 + X_eq2 + X_eq3 + X_eq4)
missing_features = [f for f in all_features if f not in df_model.columns]
if missing_features:
    print(f"Warning: The following features are missing from the dataframe: {missing_features}")
    print("Available columns containing 'density' or 'center':")
    density_center_cols = [col for col in df_model.columns if 'density' in col.lower() or 'center' in col.lower()]
    print(density_center_cols)

print("Starting model training...")
print("="*50)

# Model configurations
model_configs = [
    # Spatial models
    # {'features': X_eq0, 'name': 'Model 0', 'spatial': True, 
    #  'description': 'Full feature set with spatial'},
    
    {'features': X_eq1, 'name': 'Model 1 - Neighborhood - SEBS rates', 'spatial': True, 'outcome':'SEBS participation rate (%)',
     'description': 'Reduced feature set with spatial'},
    
    {'features': X_eq1, 'name': 'Model 1 - Neighborhood - Raw rates', 'spatial': True, 'outcome':'Participation rate (%)',
     'description': 'Reduced feature set with spatial - Raw participation rates'},
    
    # {'features': X_eq3, 'name': 'Model 3', 'spatial': True, 
    #  'description': 'Center density only with spatial'},
    
    # {'features': X_eq4, 'name': 'Model 4', 'spatial': False,  # Fixed: X_eq4 doesn't work with spatial
    #  'description': 'Nearest center only (non-spatial)'},
    
    # Non-spatial models
    {'features': X_eq1, 'name': 'Model 1 - Neighborhood - SEBS rates - Not spatial', 'spatial': False, 'outcome':'SEBS participation rate (%)',
     'description': 'SES only without spatial'},
    
    # {'features': X_eq3, 'name': 'Model 3 - Not spatial', 'spatial': False, 
    #  'description': 'Center density only without spatial'},
    
    # {'features': X_eq4, 'name': 'Model 4 - Not spatial', 'spatial': False, 
    #  'description': 'Nearest center only without spatial'},
]

# Train all models
models = {}
results = {}

for i, config in enumerate(model_configs):
    print(f"\nTraining {config['name']}: {config['description']}")
    print(f"Features: {config['features']}")
    print(f"Spatial: {config['spatial']}")
    print("-" * 40)
    
    try:
        # Verify feature availability
        missing_features = [f for f in config['features'] if f not in df_modelling.columns]
        if missing_features:
            print(f"Warning: Missing features {missing_features} for {config['name']}")
            print("Skipping this model...")
            continue
            
        # Train model
        model, gwr_df, shap_values = train_xgboost(
            gwr_df=df_modelling, 
            X_eq=config['features'], 
            y_col=config['outcome'], 
            optimize=optimize, 
            space=space, 
            n_evals=n_evals, 
            file_prefix=config['name'],
            include_spatial=config['spatial'], 
            plot_figures=True
        )
        
        # Store results
        models[config['name']] = {
            'model': model,
            'dataframe': gwr_df,
            'shap_values': shap_values,
            'config': config
        }
        
        print(f"✓ {config['name']} completed successfully")
        
    except Exception as e:
        print(f"✗ Error training {config['name']}: {str(e)}")
        import traceback
        print("Full error traceback:")
        traceback.print_exc()
        continue

print("\n" + "="*50)
print("Model training completed!")
print(f"Successfully trained {len(models)} models:")
for name in models.keys():
    print(f"  - {name}")

# Optionally, extract individual model results for backward compatibility
if 'Model 0' in models:
    model0 = models['Model 0']['model']
    df_model0 = models['Model 0']['dataframe'] 
    shap_values_model0 = models['Model 0']['shap_values']

if 'Model 1' in models:
    model1 = models['Model 1']['model']
    df_model1 = models['Model 1']['dataframe']
    shap_values_model1 = models['Model 1']['shap_values']

if 'Model 2' in models:
    model2 = models['Model 2']['model']
    df_model2 = models['Model 2']['dataframe']
    shap_values_model2 = models['Model 2']['shap_values']

if 'Model 3' in models:
    model3 = models['Model 3']['model']
    df_model3 = models['Model 3']['dataframe']
    shap_values_model3 = models['Model 3']['shap_values']

if 'Model 4' in models:
    model4 = models['Model 4']['model']
    df_model4 = models['Model 4']['dataframe']
    shap_values_model4 = models['Model 4']['shap_values']

if 'Model 2 - Not spatial' in models:
    model2_bis = models['Model 2 - Not spatial']['model']
    df_model2_bis = models['Model 2 - Not spatial']['dataframe']
    shap_values_model2_bis = models['Model 2 - Not spatial']['shap_values']

if 'Model 3 - Not spatial' in models:
    model3_bis = models['Model 3 - Not spatial']['model']
    df_model3_bis = models['Model 3 - Not spatial']['dataframe']
    shap_values_model3_bis = models['Model 3 - Not spatial']['shap_values']

if 'Model 4 - Not spatial' in models:
    model4_bis = models['Model 4 - Not spatial']['model']
    df_model4_bis = models['Model 4 - Not spatial']['dataframe']
    shap_values_model4_bis = models['Model 4 - Not spatial']['shap_values']